In [1]:
import os

In [2]:
%pwd

'h:\\Projects\\mlops\\first\\pizza-steak-end-to-end\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'h:\\Projects\\mlops\\first\\pizza-steak-end-to-end'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    data_dir: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int
    params_validation_split: float

In [6]:
from PizzaSteakClassifier.constants import *
from PizzaSteakClassifier.utils.common import read_yaml_file, create_directories


In [7]:
X = read_yaml_file(CONFIG_FILE_PATH)

[2024-01-13 15:03:18,697 - INFO - common - YAML file loaded successfully: config\config.yaml]


In [8]:
X.base_model_preparation.root_dir

'artifacts/base_model_preparation'

In [9]:
class ConfigurationManager:
    def __init__(self,
                 config_file_path = CONFIG_FILE_PATH,
                 params_file_path = PARAMS_FILE_PATH):
        
        self.config = read_yaml_file(config_file_path)
        self.params = read_yaml_file(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.base_model_preparation
        params = self.params

        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir = Path(config.root_dir),
            base_model_path = Path(config.base_model_path),
            updated_base_model_path = Path(config.updated_base_model_path),
            data_dir = Path(self.config.data_ingestion.unzip_dir),
            params_image_size = params.image_size,
            params_learning_rate = params.learning_rate,
            params_include_top = params.include_top,
            params_weights = params.weights,
            params_classes = params.classes,
            params_validation_split=params.validation_split
        )

        return prepare_base_model_config

In [10]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

[2024-01-13 15:03:27,143 - WARNING - module_wrapper - From h:\Projects\mlops\first\pizza-steak-end-to-end\.venv\Lib\site-packages\keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.
]


In [11]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config
    
    def download_base_model(self):
        self.model = tf.keras.applications.MobileNetV2(input_shape=tuple(self.config.params_image_size),
                                                       include_top=self.config.params_include_top,
                                                       weights=self.config.params_weights)
        self.save_base_model(path = self.config.base_model_path,
                             model = self.model)
    
    
    
    def prepare_full_model(self):
        base_model = self.model
        base_model.trainable = False

        data_augmentation = tf.keras.Sequential([
        tf.keras.layers.RandomFlip('horizontal'),
        tf.keras.layers.RandomRotation(0.2),
        ])

        preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

        # image_batch, label_batch = next(iter(self.train_dataset))
        # feature_batch = base_model(image_batch)

        global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
        # feature_batch_average = global_average_layer(feature_batch)

        # rescale = tf.keras.layers.Rescaling(1./127.5, offset=-1)

        prediction_layer = tf.keras.layers.Dense(1)
        # prediction_batch = prediction_layer(feature_batch_average)

        inputs = tf.keras.Input(shape=(160, 160, 3))
        x = data_augmentation(inputs)
        x = preprocess_input(x)
        x = base_model(x, training=False)
        x = global_average_layer(x)
        x = tf.keras.layers.Dropout(0.2)(x)
        outputs = prediction_layer(x)
        full_model = tf.keras.Model(inputs, outputs)

        full_model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=self.config.params_learning_rate),
            loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
            metrics=[tf.keras.metrics.BinaryAccuracy(threshold=0, name='accuracy')]
        )

        full_model.summary()

        return full_model
    
    @staticmethod
    def save_base_model(path: Path, model: tf.keras.models.Model):
        model.save(path)
    
    def update_base_model(self):
        self.full_model = self.prepare_full_model()
        self.save_base_model(path = self.config.updated_base_model_path,
                             model = self.full_model)


In [12]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.download_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2024-01-13 15:03:30,282 - INFO - common - YAML file loaded successfully: config\config.yaml]
[2024-01-13 15:03:30,286 - INFO - common - YAML file loaded successfully: params.yaml]
[2024-01-13 15:03:30,288 - INFO - common - Directory created: artifacts]
[2024-01-13 15:03:30,288 - INFO - common - Directory created: artifacts/base_model_preparation]
[2024-01-13 15:03:30,546 - WARNING - module_wrapper - From h:\Projects\mlops\first\pizza-steak-end-to-end\.venv\Lib\site-packages\keras\src\backend.py:1398: The name tf.executing_eagerly_outside_functions is deprecated. Please use tf.compat.v1.executing_eagerly_outside_functions instead.
]
[2024-01-13 15:03:30,698 - WARNING - module_wrapper - From h:\Projects\mlops\first\pizza-steak-end-to-end\.venv\Lib\site-packages\keras\src\layers\normalization\batch_normalization.py:979: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.
]
9406464/9406464 [==============================] - 3s 0us/step
[2024

h:\Projects\mlops\first\pizza-steak-end-to-end\.venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 160, 160, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 160, 160, 3)       0         
                                                                 
 tf.math.truediv (TFOpLambd  (None, 160, 160, 3)       0         
 a)                                                              
                                                                 
 tf.math.subtract (TFOpLamb  (None, 160, 160, 3)       0         
 da)                                                             
                                                                 
 mobilenetv2_1.00_160 (Func  (None, 5, 5, 1280)        2257984   
 tional)                                                         
                                                             